In [1]:
from models_train.IQAmodel import IQAModel
import os
import numpy as np
import random
import torch
from argparse import ArgumentParser
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import resize, to_tensor, normalize
import torch.nn.functional as F
from PIL import Image
import h5py
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt


env: CUDA_VISIBLE_DEVICES=2


In [14]:
def denorm(mmin, mmax, df):
    df['clear_val'] = df['clear_val'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=2'] = df['attacked_val_eps=2'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=4'] = df['attacked_val_eps=4'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=6'] = df['attacked_val_eps=6'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=8'] = df['attacked_val_eps=8'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=10'] = df['attacked_val_eps=10'].transform(lambda x: x*(mmax-mmin)+mmin)

In [15]:
def return_weight_path(csv_path):
    splt = csv_path.split('+')
    activation = splt[-1].rsplit('_', 1)[0]
    arch = splt[0].rsplit('_', 1)[-1]
    # print(activation, arch)
    format_str = 'activation={}-Linearity-{}-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664' \
                .format(activation, arch)
    if 'gr' in csv_path:
        format_str += f'-gr=True'
    if 'cayley_pool' in csv_path:
        format_str += f'-clp=True'
    if '0.1_l2' in csv_path:
        format_str += '+prune=0.1l2_lr=1e-06_e=5_iters=1'
    return format_str

In [25]:
path = "KonIQ-10k_resnet101+relu_PGD=1.csv"
CSV_PATH = "/home/28i_mel@lab.graphicon.ru/mnt/dione/28i_mel"
weights_path = "activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664-clp=True+prune=0.1l2_lr=1e-06_e=5_iters=1"

In [39]:

lib = Path(os.path.join(CSV_PATH, 'Linearity-results'))
it = Path(lib).iterdir()
for csv_path in it:
    # print(csv_path)
    df = pd.read_csv(csv_path, index_col=False)
    print(csv_path.name)
    weight_path = return_weight_path(str(csv_path.name))
    print(weight_path)
    ckpt = torch.load(os.path.join(CSV_PATH, 'Linearity-ckpt', weight_path),
                      map_location='cpu')
    mmin = ckpt['min']
    mmax = ckpt['max']
    denorm(mmin, mmax, df)
    print(df['clear_val'].min(), df['clear_val'].max())
    df = df.drop(['Unnamed: 0'], axis=1)
    save_path = os.path.join(CSV_PATH, 'denorm_Linearity', csv_path.name)
    df.to_csv(save_path, index=False)

    
    

KonIQ-10k_resnet101+relu_PGD=1.csv
activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664


/tmp/ipykernel_2183491/494465267.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, 'Linearity-ckpt', weight_path),


5.843866884924891 80.04117103754798
KonIQ-10k_resnet101+relu_PGD=3.csv
activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664


/tmp/ipykernel_2183491/494465267.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, 'Linearity-ckpt', weight_path),


5.843866884924891 80.04117103754798
KonIQ-10k_resnet101+relu_PGD=5.csv
activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664


...

10.846460615256444 83.91715652752995
NIPS_resnet101+gr+relu_PGD=8.csv
activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664-gr=True


/tmp/ipykernel_2183491/494465267.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, 'Linearity-ckpt', weight_path),


34.89095496873453 96.72789788338484
KonIQ-10k_resnet101+gr+relu_PGD=8.csv
activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664-gr=True


/tmp/ipykernel_2183491/494465267.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, 'Linearity-ckpt', weight_path),


0.4096136486253119 81.59719982637877


In [21]:
ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))

/tmp/ipykernel_1900921/4219770622.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))


In [22]:
(ckpt['min'], ckpt['max'])

(np.float64(11.457558137907661), np.float64(85.37204118074094))

In [26]:
df = pd.read_csv(os.path.join(CSV_PATH, "Linearity-results", path))

In [27]:
df

,Unnamed: 0,image_name,clear_val,attacked_val_eps=2,attacked_val_eps=4,attacked_val_eps=6,attacked_val_eps=8,attacked_val_eps=10
0,0,10007357496.jpg,70.315854,83.195828,81.691779,79.288533,76.818263,74.222213
1,1,10020766793.jpg,74.534388,85.327396,83.533678,81.307683,78.834983,75.840654
2,2,10020891105.jpg,50.595274,75.034523,75.083722,73.295596,71.884868,70.701743
3,3,10022757465.jpg,67.659917,76.097159,76.046806,75.179123,74.163184,72.749575
4,4,10039534103.jpg,62.029985,66.565415,63.524834,60.966525,60.931905,60.699611
...,...,...,...,...,...,...,...,...
2010,2010,9936084895.jpg,58.194785,73.561303,71.053460,69.429730,68.726635,67.899999
2011,2011,9977056676.jpg,45.765210,67.099404,66.425987,64.738204,63.951925,63.853729
2012,2012,9977191265.jpg,63.821208,77.215777,74.920513,74.095299,74.539897,74.087674
2013,2013,9984535544.jpg,68.005197,78.854500,77.656356,76.151656,74.886644,74.011969


In [28]:
(df['clear_val'].min(), df['clear_val'].max())

(np.float64(5.843866884924886), np.float64(80.04117103754798))